In [1]:
import pandas as pd

In [67]:
df = pd.read_csv("glassdoor_jobs.csv")

In [70]:
df.head()

Unnamed: 0                  Job Title              Salary Estimate  \
0           0             Data Scientist   $53K-$91K (Glassdoor est.)   
1           1  Healthcare Data Scientist  $63K-$112K (Glassdoor est.)   
2           2             Data Scientist   $80K-$90K (Glassdoor est.)   
3           3             Data Scientist   $56K-$97K (Glassdoor est.)   
4           4             Data Scientist  $86K-$143K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Data Scientist\nLocation: Albuquerque, NM\nEdu...     3.8   
1  What You Will Do:\n\nI. General Summary\n\nThe...     3.4   
2  KnowBe4, Inc. is a high growth information sec...     4.8   
3  *Organization and Job ID**\nJob ID: 310709\n\n...     3.8   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

                                 Company Name         Location  \
0                      Tecolote Research\n3.8  Albuquerque, NM   
1  University of Maryland Medical System\n3.4    Linthicum, MD   
2                                KnowBe4\n4.8   Clearwater, FL   
3                                   PNNL\n3.8     Richland, WA   
4                     Affinity Solutions\n2.9     New York, NY   

     Headquarters                    Size  Founded   Type of ownership  \
0      Goleta, CA   501 to 1000 employees     1973   Company - Private   
1   Baltimore, MD        10000+ employees     1984  Other Organization   
2  Clearwater, FL   501 to 1000 employees     2010   Company - Private   
3    Richland, WA  1001 to 5000 employees     1965          Government   
4    New York, NY     51 to 200 employees     1998   Company - Private   

                           Industry                        Sector  \
0               Aerospace & Defense           Aerospace & Defense   
1  Health Care Services & Hospitals                   Health Care   
2                 Security Services             Business Services   
3                            Energy  Oil, Gas, Energy & Utilities   
4           Advertising & Marketing             Business Services   

                            Revenue  \
0         $50 to $100 million (USD)   
1            $2 to $5 billion (USD)   
2        $100 to $500 million (USD)   
3  $500 million to $1 billion (USD)   
4          Unknown / Non-Applicable   

                                         Competitors  hourly_salary  \
0                                                 -1              0   
1                                                 -1              0   
2                                                 -1              0   
3  Oak Ridge National Laboratory, National Renewa...              0   
4               Commerce Signals, Cardlytics, Yodlee              0   

   employer_provided_salary  min_salary  max_salary  avg_salary  
0                         0          53          91        72.0  
1                         0          63         112        87.5  
2                         0          80          90        85.0  
3                         0          56          97        76.5  
4                         0          86         143       114.5

# Salary data

In [69]:
# salary
df = df[df['Salary Estimate']!='-1']   # remove rows with -1 in salary estimate
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])            # remove (Glassdoor est.)
salary_nums = salary.apply(lambda x: x.replace('K','').replace('$',''))    # remove K and $

df['hourly_salary'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)   # mark rows which have salary per hour
df['employer_provided_salary'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)  # mark rows which have employer provided salary

salary_clean = salary_nums.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))  # remove 'per hour' and 'employer provided salary' from salary

df['min_salary'] = salary_clean.apply(lambda x: int(x.split('-')[0]))       # take the min salary amount from salary range
df['max_salary'] = salary_clean.apply(lambda x: int(x.split('-')[1]))       # max salary amount from salary range
df['avg_salary'] = (df.min_salary + df.max_salary)/2                        

# Company Name

In [87]:
# extract only the name of the company, remove the rating
df['company_text'] = df.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'][:-4],axis=1)

# Location (State) of the job

In [72]:
# extract the state from the location of the job
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])
#df.job_state.value_counts()   # describes how many jobs are in each state

In [73]:
# check if headquarters is in the same state as the job location
df['same_state'] = df.apply(lambda x: 1 if x.Location==x.Headquarters else 0, axis=1)

# How old is the company

In [74]:
# if entry is -1 then keep it -1, else 2020-founded year
df['company_age'] = df.Founded.apply(lambda x: x if x<1 else 2020-x)    

# Skills in job description

In [80]:
# Look for python, R studio, excel, spark, aws

df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
#df.python.value_counts()
    
df['r_studio'] = df['Job Description'].apply(lambda x: 1 if 'r-studio' in x.lower() or 'r studio' in x.lower() else 0)
df.r_studio.value_counts()

df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
#df.spark.value_counts()

df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
#df.aws.value_counts()

df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
#df.excel.value_counts()

# Make csv of the cleaned data

In [90]:
df_out = df.drop(['Unnamed: 0'], axis=1)

In [91]:
df_out.to_csv('salary_data_cleaned_final.csv',index=False)

In [92]:
data = pd.read_csv('salary_data_cleaned_final.csv')
data.head()

Job Title              Salary Estimate  \
0             Data Scientist   $53K-$91K (Glassdoor est.)   
1  Healthcare Data Scientist  $63K-$112K (Glassdoor est.)   
2             Data Scientist   $80K-$90K (Glassdoor est.)   
3             Data Scientist   $56K-$97K (Glassdoor est.)   
4             Data Scientist  $86K-$143K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Data Scientist\nLocation: Albuquerque, NM\nEdu...     3.8   
1  What You Will Do:\n\nI. General Summary\n\nThe...     3.4   
2  KnowBe4, Inc. is a high growth information sec...     4.8   
3  *Organization and Job ID**\nJob ID: 310709\n\n...     3.8   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

                                 Company Name         Location  \
0                      Tecolote Research\n3.8  Albuquerque, NM   
1  University of Maryland Medical System\n3.4    Linthicum, MD   
2                                KnowBe4\n4.8   Clearwater, FL   
3                                   PNNL\n3.8     Richland, WA   
4                     Affinity Solutions\n2.9     New York, NY   

     Headquarters                    Size  Founded   Type of ownership  ...  \
0      Goleta, CA   501 to 1000 employees     1973   Company - Private  ...   
1   Baltimore, MD        10000+ employees     1984  Other Organization  ...   
2  Clearwater, FL   501 to 1000 employees     2010   Company - Private  ...   
3    Richland, WA  1001 to 5000 employees     1965          Government  ...   
4    New York, NY     51 to 200 employees     1998   Company - Private  ...   

  avg_salary                           company_text job_state same_state  \
0       72.0                      Tecolote Research        NM          0   
1       87.5  University of Maryland Medical System        MD          0   
2       85.0                                KnowBe4        FL          1   
3       76.5                                   PNNL        WA          1   
4      114.5                     Affinity Solutions        NY          1   

   company_age  python  r_studio  spark  aws excel  
0           47       1         0      0    0     1  
1           36       1         0      0    0     0  
2           10       1         0      1    0     1  
3           55       1         0      0    0     0  
4           22       1         0      0    0     1  

[5 rows x 28 columns]